In [25]:
import time
from multiprocessing import Pool, freeze_support
from pathlib import Path

from tinydb import where
from tqdm.auto import tqdm
from default_logger.defaultLogger import defaultLogger
from scrapper.brand.asos.Asos import Asos
from scrapper.brand.asos.helper.database.dbhelper import list_dbs_by_category
from scrapper.brand.asos.helper.download.DownloadHelper import DownloadHelper
from scrapper.util.io import Json_DB
from scrapper.util.web.dynamic import driver as d_driver
from scrapper.brand.asos.helper.download.AsosPaths import AsosPaths
from scrapper.util.list import distinct_list_of_dicts, flatten
CATEGORIES = [
    {"name": "schuhe", "includes": ["shoe"], "excludes": []},
    {"name": "hose", "includes": ["short", "jeans", "leggings", "trousers"], "excludes": []},
    {"name": "shirt", "includes": ["shirt", "skirt", "blazer", "top"], "excludes": []},
    {"name": "pullover", "includes": ["pullover", "cardigans"], "excludes": []},
    {"name": "jacke", "includes": ["coat"], "excludes": []},
    {"name": "kleid", "includes": ["dresses"], "excludes": ["kleidung"]},
    {"name": "anzug", "includes": ["suit", "overalls"], "excludes": []}
]


In [26]:

#asos = Asos(d_driver(headless=False))

categories_db_path = brand_path.get_category_db_base_path()
entries_db_path = brand_path.get_entries_db_base_path()

entries = list_dbs_by_category(entries_db_path, CATEGORIES)

In [27]:
entries.keys()

dict_keys(['hose', 'schuhe', 'shirt'])

In [57]:
import numpy as np
from random import shuffle
from scrapper.util.list import flatten, idx_self_reference

from scrapper.brand.mango.helper.database.filter import validate_triplets

In [99]:
def clean_entry(entry):
    return {"id": entry["url"], "images": [{"path": brand_path.relative_image_path_from_url(img["url"]), "view": img["description"]} for img in entry["images"]]}

def build_positive_anchors(entry): #dup
    images_anchor, anchor_id = entry["images"], entry["id"]
    positive_idxs = _random_references(len(images_anchor))
    images_positives = [images_anchor[idx] for idx in positive_idxs]
    return [{"anchor": {**a, "id": anchor_id}, "positive": {**p, "id": anchor_id}} for (a, p) in
            zip(images_anchor, images_positives)]

def preprocess_negative_entries(anchor_items): #dup
    negative_items = np.array(anchor_items)
    shuffle(negative_items)

    def __clean_entry(entry):
        return [{**x, 'id': entry["id"]} for x in entry["images"]]

    assert len(anchor_items) == len(negative_items)
    negative_items = [__clean_entry(x) for x in negative_items]
    negative_flat = flatten(negative_items)

    shuffle(negative_flat)

    return negative_flat

def preprocess_positive_entries(entries):
    return flatten([build_positive_anchors(entry) for entry in entries])

def _random_references(num_idxs, max_retries=150): #dup
    random_idxs = list(range(num_idxs))
    for _ in range(max_retries):
        shuffle(random_idxs)  # in place
        if not idx_self_reference(random_idxs):
            break
    return np.array(random_idxs)  # copy -> None otherwise


In [100]:
def preprocess_negative_entries(anchor_items):
    negative_items = np.array(anchor_items)

    shuffle(negative_items)
    assert len(anchor_items) == len(negative_items)
    return negative_items
    

In [79]:
flatten_item = lambda item: {"id": item["id"]}
anchor_items[0]

TypeError: 'map' object is not subscriptable

In [80]:
category_name = "hose"
entries_by_cat_distinct = distinct_list_of_dicts(flatten([Json_DB(x).all() for x in entries[category_name]]), key="url")

anchor_items = list(map(clean_entry, entries_by_cat_distinct))

In [101]:
negative_items = preprocess_negative_entries(anchor_items)
positive_items = preprocess_positive_entries(anchor_items)
data = []

In [103]:
positive_items[0]

{'anchor': {'path': WindowsPath('F:/workspace/fascrapper/scrap_results/asos/pics/products/pull-bear-premium-ripped-super-skinny-jeans-in-blue/201033636-1-blue'),
  'view': 'Pull & Bear premium ripped super skinny jeans in blue, 1 of 4',
  'id': 'https://www.asos.com/pullbear/pull-bear-premium-ripped-super-skinny-jeans-in-blue/prd/201033636?colourwayid=201033637&cid=4208'},
 'positive': {'path': WindowsPath('F:/workspace/fascrapper/scrap_results/asos/pics/products/pull-bear-premium-ripped-super-skinny-jeans-in-blue/201033636-3'),
  'view': 'Pull & Bear premium ripped super skinny jeans in blue, 3 of 4',
  'id': 'https://www.asos.com/pullbear/pull-bear-premium-ripped-super-skinny-jeans-in-blue/prd/201033636?colourwayid=201033637&cid=4208'}}

In [87]:
negative_items = preprocess_negative_entries(anchor_items)

TypeError: len() of unsized object

In [56]:
anchor_positive_items = flatten(list(map(build_positive_anchors, anchor_items)))
negative_items = preprocess_negative_entries(anchor_items)
data = []

for row in anchor_positive_items:
    anchor_id, positive_id = row["anchor"]["id"], row["positive"]["id"]
    negative_anchor = None
    
    while not negative_anchor:
        possible_n_anchor = negative_items.pop()
        if possible_n_anchor["id"] != anchor_id:
            negative_anchor = possible_n_anchor
        elif len(negative_items) < 2:  # for the random chance of only the same id beeing left
            break
        else:
            negative_items.insert(0, possible_n_anchor)
            
    data.append({**row, "negative": negative_anchor})